In [1]:
import numpy as np
from utils_predictions_manipulation import*
from utils_nans_manipulation import*
from cross_validation import*
from utils_data_loading import*
from utils_features_manipulation import*
import random

from time import time

## Load data

In [2]:
traindata,_ = load_data('Data/train.csv')
X_total, Y_total = structure_data(traindata)

# Replacing undefined data with NaNs
X_nans = replace_bad_data_with_nans(X_total, -999)

X_nans, col = replace_nans_with_median(X_nans, threshold=0.5)

#X_nans = X_nans + np.random.rand(X_nans.shape[0], X_nans.shape[1])/10000

In [3]:
#Baseline
dtmp_tr,dtmp_te = cross_validation(Y_total, X_nans, k_fold=4, seed=1, function_name='least_squares')

dtmp_tr

0.7338866666666667

## Setup genetic pool

In [69]:
degree_test = [1/30, 1/20, 1/10, 1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5, 10, 20, 30]
deg_all = all_combinations_list(degree_test)


In [70]:
index_test = list(range(1,X_nans.shape[1]))
index_remove = [9,11,12,14,15,17,21,22]
index_test = list(set(index_test)-set(index_remove))
ind_all = all_combinations_list(index_test)

In [71]:
tot = 1*60*60 # 6h runtime
size = 100
days = int(tot/size)
print(days)

36


In [72]:
champ_surv = 5 #Can be greater than size
pop_surv = 15
mutations = 5

complexity_deg = 575
complexity_ind = len(ind_all)-1

deg_all2 = [deg_all[0:complexity_deg] for i in range(X_nans.shape[1])] #Store as chromosomes

#Initialize

#deg_pool = deg_all2.copy()

ind_pool = ind_all[0:complexity_ind]
champions = []

start = time()
for day in range(days):
    print("\n Day:", day)
    
    #Generate population
    population = []
    for i in range(size):
        expression = random.choice(ind_pool)
        genes = [random.choice(chromosome) for chromosome in deg_pool]
        population.append([expression, genes])
    
    #Test reproductive success
    success = []
    for individual in population:
        X_poly = build_poly_index(X_nans, individual[0], individual[1])
        dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
        success.append(dtmp_tr)
        print(len(individual[0]), len(individual[1]), dtmp_tr)
    print("Average", np.mean(success))
    
    #Select best performing member for posteriory
    champions.append([max(success), population[success.index(max(success))]])
    
    #Select top 10% individuals
    survive = np.argsort(success)[-pop_surv:]
    survivors = [population[live] for live in survive]
    
    #Extract their features
    ind_pool = [individual[0] for individual in survivors]
    deg_pool = [[individual[1][i] for individual in survivors] for i in range(len(deg_pool))].copy()
    
    #Add fresh material to the gene pool
    for i in range(mutations):
        for chromosome in deg_pool:
            chromosome.append(random.choice(deg_all2[0]))
        ind_pool.append(random.choice(ind_all[0:complexity_ind]))
    
    #Select best champions:
    if len(champions)>champ_surv:
        best_champs = np.argsort([ind[0] for ind in champions])[-champ_surv:]
        champions = [champions[i] for i in best_champs]
    print(len(champions))
    
    #Add champion values to pool
    ind_pool.extend([ind[1][0] for ind in champions])
    for i, chromosome in enumerate(deg_pool):
        individual_genes = [ind[1][1] for ind in champions]
        chromosome.extend([genes[i] for genes in individual_genes])
    #print(len(deg_pool[0]), deg_pool[0])
    
    end = time()
    print(end-start)


 Day: 0
6 24 0.7732680000000001
6 24 0.7531826666666667
6 24 0.7569666666666667
7 24 0.7509813333333334
5 24 0.769164
10 24 0.7913493333333333
8 24 0.791188
6 24 0.744872
9 24 0.7562866666666668
9 24 0.7649093333333334
7 24 0.7675746666666666
9 24 0.7899866666666667
8 24 0.7668053333333333
7 24 0.7892600000000001
9 24 0.7618586666666667
5 24 0.743004
6 24 0.7828346666666666
8 24 0.7631479999999999
5 24 0.750232
5 24 0.7552613333333333
12 24 0.7831626666666667
6 24 0.755644
10 24 0.75852
9 24 0.7661466666666666
7 24 0.760772
5 24 0.7857666666666666
7 24 0.79122
9 24 0.7888293333333334
8 24 0.778204
7 24 0.7733026666666667
6 24 0.7891333333333332
5 24 0.7678053333333333
5 24 0.7411920000000001
6 24 0.7674813333333332
8 24 0.763152
8 24 0.788272
11 24 0.7894306666666666
7 24 0.7835146666666667
5 24 0.761012
7 24 0.7914626666666668
8 24 0.784924
10 24 0.760796
9 24 0.7561986666666667
6 24 0.762016
8 24 0.7899146666666667
5 24 0.7887973333333334
8 24 0.7750333333333334
6 24 0.753912
10 24 

9 24 0.7945146666666667
9 24 0.7631893333333333
9 24 0.7925826666666667
7 24 0.738996
9 24 0.7933053333333334
9 24 0.7923239999999999
9 24 0.7761226666666666
9 24 0.79454
9 24 0.792244
7 24 0.7707813333333333
9 24 0.7931346666666667
9 24 0.7587626666666667
9 24 0.7587053333333333
9 24 0.7924493333333333
9 24 0.791168
9 24 0.7573653333333333
9 24 0.7811706666666667
9 24 0.7925506666666667
9 24 0.7946773333333333
10 24 0.7882333333333333
9 24 0.763604
10 24 0.7923520000000001
10 24 0.79218
9 24 0.7878080000000001
9 24 0.7932186666666666
7 24 0.7682866666666666
9 24 0.7905973333333334
9 24 0.7932373333333333
9 24 0.7653986666666667
9 24 0.7905626666666667
9 24 0.7231173333333334
Average 0.7812085600000002
4
497.90756487846375

 Day: 4
9 24 0.793948
9 24 0.7765453333333333
9 24 0.7929493333333333
9 24 0.775652
9 24 0.7919746666666666
9 24 0.7605519999999999
9 24 0.7904760000000001
9 24 0.7595893333333333
9 24 0.7896653333333333
9 24 0.7896026666666667
9 24 0.7935026666666667
9 24 0.7945546

9 24 0.7949240000000001
9 24 0.7757120000000001
9 24 0.7948306666666667
9 24 0.7625053333333334
9 24 0.794492
9 24 0.77688
9 24 0.7888093333333333
9 24 0.7945026666666667
11 24 0.7715013333333334
9 24 0.776524
9 24 0.792852
9 24 0.7941173333333333
8 24 0.7700853333333333
9 24 0.7679266666666666
8 24 0.7684413333333333
9 24 0.7951573333333333
9 24 0.793156
9 24 0.793404
9 24 0.7924626666666666
9 24 0.7939773333333333
9 24 0.7933399999999999
9 24 0.7758626666666666
7 24 0.7561773333333333
9 24 0.7929560000000001
9 24 0.7941986666666667
9 24 0.7770453333333334
9 24 0.7603973333333334
9 24 0.7943386666666666
9 24 0.7699
7 24 0.7867280000000001
9 24 0.7947439999999999
9 24 0.7908626666666667
9 24 0.7947453333333333
9 24 0.7953786666666666
9 24 0.7947359999999999
9 24 0.7902706666666666
7 24 0.7223013333333335
9 24 0.7943439999999999
9 24 0.7631493333333335
9 24 0.794484
9 24 0.7931813333333333
9 24 0.7552266666666667
9 24 0.7730586666666667
9 24 0.7925760000000001
9 24 0.774556
7 24 0.72422

9 24 0.7945986666666667
9 24 0.7952346666666666
7 24 0.7629840000000001
9 24 0.7932853333333334
9 24 0.7728666666666667
9 24 0.794152
9 24 0.7756466666666666
9 24 0.7952106666666666
6 24 0.7417186666666666
9 24 0.7613746666666668
9 24 0.7949973333333333
9 24 0.795312
9 24 0.7942546666666668
9 24 0.7952866666666667
9 24 0.772884
9 24 0.7837626666666667
9 24 0.7941426666666667
9 24 0.7764293333333332
9 24 0.7938493333333333
9 24 0.7540239999999999
9 24 0.7935053333333333
9 24 0.7868546666666667
9 24 0.7950773333333333
9 24 0.79522
9 24 0.7702213333333332
9 24 0.7948560000000001
9 24 0.7950453333333334
9 24 0.7948946666666666
9 24 0.7916266666666667
9 24 0.7950453333333334
9 24 0.7938493333333333
6 24 0.7430973333333333
9 24 0.7741666666666667
9 24 0.7951266666666666
9 24 0.7951079999999999
9 24 0.7942453333333334
9 24 0.795164
9 24 0.79466
9 24 0.7943306666666667
9 24 0.794496
9 24 0.7877666666666667
9 24 0.7947453333333334
9 24 0.7947186666666667
9 24 0.7950933333333334
9 24 0.741776
6 

9 24 0.777864
9 24 0.7727240000000001
9 24 0.758444
9 24 0.7952466666666667
9 24 0.7950746666666666
9 24 0.7941586666666667
9 24 0.7953066666666666
10 24 0.7904626666666666
9 24 0.7952986666666667
9 24 0.7953933333333334
9 24 0.7674853333333334
9 24 0.7648093333333333
9 24 0.7717693333333333
9 24 0.7947453333333334
9 24 0.7939093333333334
9 24 0.7928
9 24 0.7954426666666667
9 24 0.7943466666666668
9 24 0.7631906666666668
9 24 0.7950573333333334
9 24 0.7942813333333333
8 24 0.7494626666666666
9 24 0.7941386666666667
9 24 0.7943560000000001
9 24 0.7924653333333334
10 24 0.790408
9 24 0.7955746666666667
Average 0.7835611866666666
5
1827.4412870407104

 Day: 15
7 24 0.7918306666666667
9 24 0.794612
9 24 0.7773413333333333
9 24 0.7938333333333333
7 24 0.7707146666666667
9 24 0.7952306666666666
9 24 0.7954146666666667
9 24 0.7937346666666667
9 24 0.7774933333333334
9 24 0.7949946666666667
9 24 0.7951986666666666
7 24 0.7900066666666665
9 24 0.7733159999999999
9 24 0.776876
9 24 0.79496266666

9 24 0.7942413333333334
8 24 0.7598733333333333
9 24 0.7728906666666666
6 24 0.7873586666666667
9 24 0.7955786666666665
9 24 0.7779759999999999
9 24 0.7950759999999999
6 24 0.758492
6 24 0.7908373333333334
9 24 0.79544
9 24 0.7767933333333333
9 24 0.7705573333333334
9 24 0.7726999999999999
9 24 0.7949773333333333
9 24 0.7950360000000001
9 24 0.7949693333333333
9 24 0.7930119999999999
9 24 0.795528
7 24 0.7417600000000001
9 24 0.7954186666666667
9 24 0.7519586666666667
9 24 0.7949546666666667
9 24 0.7884506666666666
9 24 0.7956533333333333
8 24 0.7616613333333333
9 24 0.7728653333333333
6 24 0.7895133333333333
9 24 0.7907973333333334
6 24 0.7605466666666667
9 24 0.795712
9 24 0.787372
7 24 0.783112
9 24 0.7937080000000001
9 24 0.7948813333333333
9 24 0.7949053333333334
9 24 0.7770079999999999
7 24 0.786696
9 24 0.790852
9 24 0.7781933333333333
9 24 0.7956479999999999
9 24 0.794832
9 24 0.795496
9 24 0.78772
9 24 0.7927626666666667
9 24 0.7778306666666667
7 24 0.784796
9 24 0.79415866666

9 24 0.764004
9 24 0.7925800000000001
9 24 0.7957026666666667
9 24 0.7564186666666667
9 24 0.755888
9 24 0.7958719999999999
9 24 0.775124
9 24 0.795492
9 24 0.7550546666666667
9 24 0.7957053333333333
9 24 0.751992
9 24 0.7954279999999999
9 24 0.7770173333333334
9 24 0.7550866666666666
9 24 0.6944573333333334
8 24 0.7868333333333333
9 24 0.7101213333333334
8 24 0.7867973333333333
9 24 0.7843293333333333
8 24 0.7883293333333333
9 24 0.7949813333333333
9 24 0.7939266666666667
9 24 0.7958746666666667
9 24 0.7941733333333333
9 24 0.7938120000000001
9 24 0.7946200000000001
9 24 0.7960173333333334
9 24 0.7938293333333334
9 24 0.7958013333333332
9 24 0.793736
9 24 0.7631026666666667
9 24 0.78878
9 24 0.7929253333333333
9 24 0.7954546666666666
9 24 0.7942746666666667
9 24 0.7956226666666666
9 24 0.7940719999999999
9 24 0.7954586666666668
8 24 0.787032
7 24 0.7542373333333333
8 24 0.7666200000000001
9 24 0.7952986666666666
9 24 0.79398
9 24 0.7957413333333333
9 24 0.7945453333333333
9 24 0.79107

7 24 0.740968
6 24 0.7902213333333333
9 24 0.795512
9 24 0.7904186666666666
9 24 0.7945693333333333
9 24 0.7942466666666667
9 24 0.794072
9 24 0.7950453333333334
6 24 0.76076
9 24 0.7923626666666667
7 24 0.7896960000000001
9 24 0.7952746666666667
9 24 0.7944066666666666
9 24 0.7881093333333333
9 24 0.7937693333333332
9 24 0.789452
9 24 0.7943666666666667
Average 0.7866556133333333
5
3111.6660709381104

 Day: 26
9 24 0.701152
9 24 0.79484
9 24 0.7863800000000001
4 24 0.7873266666666666
9 24 0.7947213333333333
9 24 0.7735599999999999
9 24 0.7948613333333333
9 24 0.7772666666666668
8 24 0.7861386666666667
9 24 0.7956453333333333
9 24 0.7947866666666666
8 24 0.7872426666666666
9 24 0.7905559999999999
9 24 0.7673813333333335
9 24 0.7953013333333333
9 24 0.7473400000000001
9 24 0.7734986666666667
9 24 0.7952653333333333
4 24 0.738548
9 24 0.79014
9 24 0.7933413333333332
4 24 0.7877786666666667
9 24 0.7750079999999999
9 24 0.786296
9 24 0.7959946666666666
9 24 0.7748613333333333
9 24 0.790108

9 24 0.794512
9 24 0.7885613333333333
5 24 0.74006
4 24 0.7282866666666666
4 24 0.731404
9 24 0.7771373333333333
9 24 0.7954266666666667
9 24 0.7954813333333333
9 24 0.7753746666666668
9 24 0.7956373333333334
9 24 0.7947146666666668
9 24 0.7604946666666667
7 24 0.785832
9 24 0.7956706666666666
9 24 0.788652
9 24 0.789608
7 24 0.7518946666666666
9 24 0.7953946666666667
9 24 0.7775733333333334
9 24 0.7951853333333334
9 24 0.7945826666666667
9 24 0.7959106666666667
9 24 0.7914266666666667
9 24 0.7889613333333333
9 24 0.7954946666666667
9 24 0.7617120000000001
9 24 0.7782746666666667
9 24 0.79008
7 24 0.738452
9 24 0.7927346666666667
9 24 0.790304
9 24 0.7932413333333335
9 24 0.7955866666666667
9 24 0.7957946666666668
9 24 0.794816
9 24 0.7829946666666666
9 24 0.795852
9 24 0.795924
9 24 0.7894639999999999
9 24 0.7959773333333333
9 24 0.7945720000000001
9 24 0.779188
9 24 0.7578866666666666
9 24 0.7957653333333333
9 24 0.795684
9 24 0.7687160000000001
9 24 0.7758066666666666
4 24 0.7405186

9 24 0.7566253333333334
9 24 0.7953946666666667
9 24 0.7948226666666667
10 24 0.7946453333333333
9 24 0.7954733333333333
9 24 0.7958666666666667
9 24 0.7543306666666667
9 24 0.7781600000000001
7 24 0.7931813333333333
9 24 0.794132
9 24 0.7951320000000001
9 24 0.7784053333333334
9 24 0.796008
9 24 0.7911440000000001
7 24 0.7566986666666666
9 24 0.7707373333333334
9 24 0.795844
9 24 0.7778973333333332
9 24 0.795768
10 24 0.7729760000000001
9 24 0.7653346666666667
9 24 0.7930200000000001
9 24 0.7920973333333334
9 24 0.795452
9 24 0.79562
9 24 0.795764
10 24 0.7792479999999999
9 24 0.7792746666666668
9 24 0.7956093333333333
9 24 0.7934906666666667
9 24 0.7957546666666666
9 24 0.7939666666666667
9 24 0.7944933333333333
9 24 0.7949933333333333
9 24 0.7943960000000001
9 24 0.7947386666666667
9 24 0.7935106666666667
9 24 0.7958386666666667
9 24 0.7899986666666666
9 24 0.7957733333333333
9 24 0.7940133333333332
9 24 0.7955920000000001
9 24 0.7914866666666667
10 24 0.7951880000000001
9 24 0.7815

In [73]:
for ind in champions:
    print(ind[0])

0.7960866666666667
0.7961133333333332
0.7961426666666667
0.7962213333333333
0.7963693333333334


In [101]:
ch_ind = champions[4][1][0]
ch_deg = champions[4][1][1]
accu = champions[4][0]

In [82]:
for ind in ch_ind:
    print(ind, ch_deg[ind])

1 (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2)
3 (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30)
5 (0.2, 0.25, 0.3333333333333333, 3, 4, 5)
7 (0.1, 0.3333333333333333, 4)
13 (1, 3, 30)
16 (0.1, 10, 30)
18 (0.05, 0.5, 30)
20 (0.3333333333333333, 3, 5)
23 (0.05, 0.2, 30)


In [95]:
ch_degm = ch_deg.copy()
ch_degm[1]

(0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2)

In [109]:
ch_degm[1][5]

2

In [110]:
#Dropping 1
for ind in ch_ind:
    
    for j in range(len(ch_degm[ind])):
        ch_degm = ch_deg.copy()
        ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][j]]))
        
        X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
        dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
        print(ind, j, dtmp_tr, dtmp_tr>=accu)
    

1 0 0.7863279999999999 False
1 1 0.7861786666666667 False
1 2 0.7859813333333333 False
1 3 0.7858333333333334 False
1 4 0.7859013333333335 False
1 5 0.787332 False
3 0 0.7943800000000001 False
3 1 0.7948906666666666 False
3 2 0.7934733333333334 False
3 3 0.7947773333333333 False
3 4 0.796068 False
5 0 0.7939160000000001 False
5 1 0.7944559999999999 False
5 2 0.795616 False
5 3 0.7963733333333333 True
5 4 0.7943106666666666 False
5 5 0.7942946666666667 False
7 0 0.7902346666666666 False
7 1 0.794332 False
7 2 0.7950066666666666 False
13 0 0.7926666666666666 False
13 1 0.7941866666666667 False
13 2 0.7934573333333332 False
16 0 0.794652 False
16 1 0.7959053333333332 False
16 2 0.7960160000000001 False
18 0 0.7959746666666667 False
18 1 0.7961 False
18 2 0.7954213333333333 False
20 0 0.7958466666666666 False
20 1 0.7954906666666667 False
20 2 0.7960813333333334 False
23 0 0.7954293333333333 False
23 1 0.795564 False
23 2 0.7952346666666666 False


In [115]:
#Dropping 2
for ind in ch_ind:
    
    for j in range(len(ch_degm[ind])):
        ch_degm = ch_deg.copy()
        ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][j]]))
        ch_ori = ch_degm[ind].copy()
        print(ind, j)
        for k in range(len(ch_degm[ind])):
            ch_degm[ind] = ch_ori.copy()
            ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][k]]))
            X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
            dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
            print("\t", ind, k, dtmp_tr, dtmp_tr>=accu)

1 0
	 1 0 0.7850813333333333 False
	 1 1 0.7847826666666666 False
	 1 2 0.786288 False
	 1 3 0.7848786666666667 False
	 1 4 0.7830066666666666 False
1 1
	 1 0 0.7853586666666666 False
	 1 1 0.785152 False
	 1 2 0.7862 False
	 1 3 0.7852266666666666 False
	 1 4 0.7847826666666666 False
1 2
	 1 0 0.7854946666666667 False
	 1 1 0.785152 False
	 1 2 0.7860333333333334 False
	 1 3 0.7853786666666667 False
	 1 4 0.7848786666666667 False
1 3
	 1 0 0.7856466666666666 False
	 1 1 0.7852266666666666 False
	 1 2 0.785844 False
	 1 3 0.7853786666666667 False
	 1 4 0.7830066666666666 False
1 4
	 1 0 0.7853586666666666 False
	 1 1 0.7854946666666667 False
	 1 2 0.7855826666666667 False
	 1 3 0.7856466666666666 False
	 1 4 0.7850813333333333 False
1 5
	 1 0 0.7855826666666667 False
	 1 1 0.7862 False
	 1 2 0.7860333333333334 False
	 1 3 0.785844 False
	 1 4 0.786288 False
3 0
	 3 0 0.7949053333333334 False
	 3 1 0.7949173333333334 False
	 3 2 0.7948973333333332 False
	 3 3 0.7951226666666666 False
3 

In [116]:
ch_degm = ch_deg.copy()
ind = 5
ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][0]]))
ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][4]]))

X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
print("\t", ind, k, dtmp_tr, dtmp_tr>=accu)


	 5 1 0.79648 True


In [119]:
ch_indm = ch_ind
for ind in ch_ind:
    ch_indm = ch_ind
    ch_indm = list(set(ch_indm) - {ind})
    X_poly = build_poly_index(X_nans, ch_indm, ch_degm)
    dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
    print("\t", ind, dtmp_tr, dtmp_tr>=accu)

	 1 0.7725000000000001 False
	 3 0.7941226666666668 False
	 5 0.7929173333333333 False
	 7 0.7956973333333333 False
	 13 0.7947893333333333 False
	 16 0.7926906666666668 False
	 18 0.794164 False
	 20 0.7952906666666666 False
	 23 0.7899160000000001 False


In [121]:
print(ch_ind, ch_degm)

(1, 3, 5, 7, 13, 16, 18, 20, 23) [(0.25, 0.3333333333333333, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.1, 0.3333333333333333, 20), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.05, 10), [0.25, 3, 4, 0.3333333333333333], (3, 10, 20), (0.1, 0.3333333333333333, 4), (0.03333333333333333, 0.2, 1), (0.1, 2, 5), (0.05, 0.1, 1), (0.2, 0.3333333333333333, 0.5, 2, 4, 10), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (1, 3, 30), (0.1, 5, 10), (0.25, 3, 30), (0.1, 10, 30), (0.2, 1, 3), (0.05, 0.5, 30), (1, 3, 4), (0.3333333333333333, 3, 5), (2, 10), (0.25, 0.3333333333333333, 1), (0.05, 0.2, 30)]


In [122]:
print(ind_pool)

[(1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 2, 16, 18, 19, 23), (2, 4, 5, 7, 13, 16, 18, 20, 23), (4, 6, 7, 19, 23), (2, 4, 7, 10, 13, 20, 23), (1, 2, 3, 6, 7, 8, 10, 13, 16, 18, 19, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 4, 5, 6, 10, 18, 19, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23)]


In [124]:
print(deg_pool)

[[(0.25, 2, 20), (0.25, 2, 3), (0.25, 2, 20), (0.25, 0.3333333333333333, 2), (0.2, 0.25, 30), (0.05, 0.5, 20), (0.25, 2, 3), (0.25, 0.3333333333333333, 2), (0.03333333333333333, 0.1, 5), (0.25, 2, 20), (0.25, 0.3333333333333333, 0.5), (0.2, 1, 4), (0.2, 1, 4), (0.25, 0.3333333333333333, 2), (0.25, 2, 3), (0.5, 4, 10), (0.1, 3, 5), (1, 5), (0.05, 1, 10), (0.5, 5), (0.25, 2, 3), (0.03333333333333333, 0.1, 5), (0.25, 2, 3), (0.03333333333333333, 0.1, 5), (0.25, 0.3333333333333333, 2)], [(0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0

# Stored selections

In [76]:
print(third_selection)

[[0.7960866666666667, [(1, 3, 5, 7, 13, 16, 18, 20, 23), [(0.25, 2, 3), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.5, 2, 20), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.05, 0.3333333333333333, 10), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.05, 0.2, 0.3333333333333333), (0.3333333333333333, 1, 2), (0.03333333333333333, 0.2, 1), (0.2, 2, 20), (0.05, 0.25, 3), (0.1, 0.3333333333333333, 4), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.25, 2, 20), (0.1, 5, 10), (0.2, 0.3333333333333333), (0.1, 10, 30), (0.03333333333333333, 2, 4), (0.05, 0.5, 30), (1, 3, 4), (0.3333333333333333, 2), (0.3333333333333333, 0.5, 5), (0.25, 0.3333333333333333, 1), (0.05, 0.2, 30)]]], [0.7961133333333332, [(1, 3, 4, 5, 6, 10, 18, 19, 23), [(0.03333333333333333, 0.1, 5), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.1, 5, 20), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.25, 2, 3, 5, 20, 30), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (2, 

In [39]:
print(second_selection)

[[0.7943933333333333, [(1, 5, 6, 10, 16, 18), [(0.25, 1, 2, 5, 30), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.2, 2, 30), (0.25, 0.5, 4, 30), (0.03333333333333333, 0.2, 0.25, 2, 4, 10), (0.1, 0.2, 1, 2, 4, 5), (0.1, 0.25, 0.5, 2, 5), (0.2, 0.5, 1, 2, 5, 30), (0.03333333333333333, 0.05, 0.3333333333333333, 1, 4, 5), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 2, 20), (0.3333333333333333, 1, 3, 10, 20, 30), (0.25, 1, 20, 30), (0.03333333333333333, 0.2, 1, 3, 10), (0.03333333333333333, 0.05, 0.5, 1, 2), (0.03333333333333333, 0.5, 3, 30), (0.1, 0.3333333333333333, 2, 5, 20), (0.1, 1, 3, 4, 10, 30), (0.05, 0.1, 0.3333333333333333, 10, 30), (0.5, 1, 4, 10, 30), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.3333333333333333, 0.5, 4, 10, 20), (0.03333333333333333, 0.25, 1, 2, 3, 10), (0.05, 0.1, 1, 3, 10, 30), (0.03333333333333333, 0.1, 2, 3, 10)]]], [0.7944106666666666, [(1, 5, 6, 10, 16, 18), [(0.03333333333333333, 0.3333333333333333, 0.5, 2), (0.0333333333333333

In [17]:
print(first_selection)

[[0.7936973333333334, [(1, 4, 5, 16), [(0.25, 0.3333333333333333, 2, 4, 10, 20), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.05, 0.5, 2, 3, 5, 10), (1, 3), (0.05, 0.1, 0.25, 0.3333333333333333, 2, 20), (0.25, 0.3333333333333333, 2, 10, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 2, 30), (0.05, 0.1, 0.2, 3, 4, 10), (0.03333333333333333, 0.25, 0.5, 1, 5, 20), (0.03333333333333333, 0.25, 1, 2, 4, 5), (0.03333333333333333, 0.05, 0.3333333333333333, 0.5, 1, 2), (0.1, 1, 3, 4, 20), (0.1, 0.25, 0.3333333333333333, 0.5, 1, 10), (0.03333333333333333, 0.05, 0.5, 3, 5, 30), (0.03333333333333333, 0.05, 0.25, 4, 5, 10), (2, 4, 20), (0.05, 4, 20), (0.25, 0.3333333333333333, 2, 20, 30), (0.05, 10, 20, 30), (0.03333333333333333, 1, 3, 5, 30), (0.03333333333333333, 0.05, 0.1, 0.2, 0.25, 0.3333333333333333, 1), (0.03333333333333333, 0.05, 0.2, 1, 4, 20), (0.03333333333333333, 0.05, 0.1, 0.3333333333333333, 2), (0.05, 0.2, 2, 3, 10, 30)]]], [0.7937160000000001, [(1, 4, 5

# Debugging

In [ ]:
def build_poly_index(tx, index_list, degree):
    """polynomial basis functions for input data tx, for all features expanded to d degree.Extended matrix is inputed np.array. Index list includes all variables that will be expanded, mostly useful to avoid expanding categorical variables"""
    # ***************************************************
    # augmented x matrix with column of 1 input
    #list of indices for features modified
    #degrees of polynomial to apply to all features irrespectively
    # ***************************************************
    added_cols = sum([(len(degree[i])-1) for i in index_list])
    xmat=np.empty((tx.shape[0], tx.shape[1] + sum([(len(degree[i])-1) for i in index_list])))
    ind = 0
    for i in range(tx.shape[1]):
        if i in index_list:
            #print("BP:", i, degree[i])
            for d in degree[i]:
                coltmp=tx[:,i]**d
                xmat[:,ind] = coltmp
                ind+=1
        else:
            coltmp=tx[:,i]
            xmat[:,ind] = coltmp
            ind+=1
    return xmat

In [ ]:
index_list = list(range(X_nans.shape[1]))

degree_test = [1/30, 1/20, 1/10, 1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5, 10, 20, 30]
degree = [degree_test for i in index_list]

index_list = [3,23]

out = build_poly_index(X_nans, index_list, degree)
out.shape

In [ ]:
X_nans[:,0]

In [ ]:
out[:,0]